In [391]:
'''
M1. Integradora


Authors:
Luis Alberto Alcántara Cabrales A01634185
Alexa Serrano Negrete A01654063
Renet de Jesús Pérez Gómez A01640555
Vicente Javier Viera Guízar A01639784

Date:
11/17/2022
'''

#Model
import agentpy as ap
import random


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

#Global variable
with open('json.txt', 'w') as f:
    f.write('First line')

f.close()

In [392]:
parameters = {
    'K': 30,
    'citySize': 60,
    'streetSize': 4,
    'steps': 250,
}

In [393]:
class CarAgent(ap.Agent):
    def setup(self):
        self.idTypeOfCar = 0

class StreetAgent(ap.Agent):
    def setup(self):
       self.idTypeOfStreet = 0

In [394]:
def initialPositionCars(cars, street, citySize):
    position = []
    '''
    If modulus is equal to 0, then it going to be in horizontal way
    if not, then it going to be in the vertical way 
    '''

    #Defining the street coordinates
    leftSide = int(((citySize - street) / 2) + 2)
    rightSide = int(leftSide + 1)
    endEnviroment = int(citySize - 1)

    print("leftSide: ", leftSide)
    print("rightSide: ", rightSide)
    print("endEnviroment: ", endEnviroment)
    #Create initial positions for the car
    
    for car in cars:         
        # Horizontal left
        if car.idTypeOfCar == 1:
            position.append((0, rightSide))

        # Horizontal right
        if car.idTypeOfCar == 2:
            position.append((endEnviroment, leftSide))

        # Vertical top
        if car.idTypeOfCar==3:
            position.append((leftSide, 0))
        
        # Vertical bottom        
        if car.idTypeOfCar==4:
            position.append((rightSide, endEnviroment))

    return position


def initialPositionStreets(streetSize, citySize):
    position = []

    #Defining the street coordinates
    rightSide = int(((citySize - streetSize) / 2) + 2)
    leftSide = int(rightSide + 1)

    for i in range(citySize):
        position.append((i, leftSide))    
    
    for i in range(citySize):
        position.append((i, rightSide))
        
    for i in range(citySize):
        position.append((rightSide, i))

    for i in range(citySize):
        position.append((leftSide, i))

    return position

In [395]:
class MyModel(ap.Model):
    def setup(self):
        #Create agents
        self.city = ap.AgentList(self, int(self.p.citySize ** 2))
        self.cars = ap.AgentList(self, self.p.K, CarAgent)
        self.streets = ap.AgentList(self, int(self.p.citySize * 4), StreetAgent)

        #Assing the type of initial position to the cars (where's going to appear for first time)
        for car in self.cars:
            car.idTypeOfCar = random.randint(1,4)

        #Assing the type of street to the streets
        aux = 0
        typeOfStreet = 1
        for street in self.streets:
            if aux < self.p.citySize:
                street.idTypeOfStreet = typeOfStreet
                aux += 1
            else:
                typeOfStreet += 1
                street.idTypeOfStreet = typeOfStreet
                aux = 1

        #Create grid
        self.area = ap.Grid(self, [self.p.citySize] * 2)
        
        #Determinate the initial position of the cars
        carsPosition = initialPositionCars(self.cars, self.p.streetSize, self.p.citySize)
        
        #Determinate the initial position of the streets
        streetsPosition = initialPositionStreets(self.p.streetSize, self.p.citySize)
        

        #Add agents to enviroment
        self.area.add_agents(self.cars, carsPosition)
        self.area.add_agents(self.streets, streetsPosition)
        
        for street in self.streets:
            [y, x] = self.area.positions[street]
            print("id street = ", street.idTypeOfStreet, "y = ", y, "x = ", x)
        
        #Defining types of agents 
        self.city.agent_type = 0
        self.cars.agent_type = 1
        self.streets.agent_type = 2
        
    def step(self):
        #Find the center of the simulation 
        rightSide = int(((self.p.citySize - self.p.streetSize) / 2) + 2)
        leftSide = int(rightSide + 1)
        
        #Move cars
        for car in self.cars:
            #Get the neighbors of the current car
            neighbors = self.area.neighbors(car)
            for neighbor in neighbors:
                #1 - Search if the current neighbor is of the "street" type
                #2 - Search if the street "match" with the current car
                if neighbor.agent_type == 2:
                    if car.idTypeOfCar == 1:
                        if neighbor.idTypeOfStreet == 1:
                            self.area.move_by(car, (1,0))
                            [y, x] = self.area.positions[car]
                            print("car id: ", car.id, " - car type street: ", car.idTypeOfCar, " - y = ", y, " x = ", x)
                            if y == rightSide and x == leftSide:
                                car.idTypeOfCar = random.choice([1,3])
                            elif y == leftSide and x == leftSide:
                                car.idTypeOfCar = random.choice([1,4])
                            break
                            

                    if car.idTypeOfCar == 2:
                        if neighbor.idTypeOfStreet == 2:
                            self.area.move_by(car, (-1,0))
                            [y, x] = self.area.positions[car]
                            print("car id: ", car.id, " - car type street: ", car.idTypeOfCar, " - y = ", y, " x = ", x)
                            if y == leftSide and x == rightSide:
                                car.idTypeOfCar = random.choice([2,4])
                            elif y == rightSide and x == rightSide:
                                car.idTypeOfCar = random.choice([2,3])
                            break

                    if car.idTypeOfCar == 3:
                        if neighbor.idTypeOfStreet == 3:
                            self.area.move_by(car, (0,1))
                            [y, x] = self.area.positions[car]
                            print("car id: ", car.id, " - car type street: ", car.idTypeOfCar, " - y = ", y, " x = ", x)
                            if y == rightSide and x == rightSide:
                                car.idTypeOfCar = random.choice([2,3])
                            elif y == rightSide and x == leftSide:
                                car.idTypeOfCar = random.choice([1,3])
                            break
                        
                    if car.idTypeOfCar == 4:
                        if neighbor.idTypeOfStreet == 4:
                            self.area.move_by(car, (0,-1))
                            [y, x] = self.area.positions[car]
                            print("car id: ", car.id, " - car type street: ", car.idTypeOfCar, " - y = ", y, " x = ", x)
                            if y == leftSide and x == leftSide:
                                car.idTypeOfCar = random.choice([1,4])
                            elif y == leftSide and x == rightSide:
                                car.idTypeOfCar = random.choice([2,4])
                            break
                elif neighbor.agent_type == 1:
                    if self.area.positions[neighbor] !=  self.area.positions[car]:
                        break

In [396]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('agent_type')
    color_dict = {0:'#000', 1:'#c1c1c1', 2:'#007', None:'#FFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic Simulation\n"
                 f"Time-step: {model.t}",
                 color="white") 

fig, ax = plt.subplots()
model = MyModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))

leftSide:  30
rightSide:  31
endEnviroment:  59
id street =  1 y =  0 x =  31
id street =  1 y =  1 x =  31
id street =  1 y =  2 x =  31
id street =  1 y =  3 x =  31
id street =  1 y =  4 x =  31
id street =  1 y =  5 x =  31
id street =  1 y =  6 x =  31
id street =  1 y =  7 x =  31
id street =  1 y =  8 x =  31
id street =  1 y =  9 x =  31
id street =  1 y =  10 x =  31
id street =  1 y =  11 x =  31
id street =  1 y =  12 x =  31
id street =  1 y =  13 x =  31
id street =  1 y =  14 x =  31
id street =  1 y =  15 x =  31
id street =  1 y =  16 x =  31
id street =  1 y =  17 x =  31
id street =  1 y =  18 x =  31
id street =  1 y =  19 x =  31
id street =  1 y =  20 x =  31
id street =  1 y =  21 x =  31
id street =  1 y =  22 x =  31
id street =  1 y =  23 x =  31
id street =  1 y =  24 x =  31
id street =  1 y =  25 x =  31
id street =  1 y =  26 x =  31
id street =  1 y =  27 x =  31
id street =  1 y =  28 x =  31
id street =  1 y =  29 x =  31
id street =  1 y =  30 x =  31
i